In [11]:
import os
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import numpy as np
import torch
import torchvision
import torch.nn as nn
import torchvision.transforms.functional as TF
from tqdm import tqdm
import torch.optim as optim
from sklearn.model_selection import KFold
from torch.utils.data import SubsetRandomSampler
from collections import OrderedDict
import matplotlib.pyplot as plt
from Network import *
from DataSet import *
from train_procedure import *
from Aug import *
from torchvision import transforms
import pickle

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
# Create the augmented dataset
create_augmented_dataset(True)

satImage_100.png: 100%|████████████████████| 100/100 [00:55<00:00,  1.79image/s]


In [13]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

## Experiment 5

In [ ]:
dict_train5 ={"save_model": False,
            "cross_val": True,
            "skip_connection": True,
            "num_epochs": 10,
            "n_splits": 2,
            "batch_size": 10,
            "scale_channel": 1,
            "dict_double_conv": {"BatchNorm": True,
                "activation": nn.ELU(inplace=True),
                "p_dropout": 0.2,
                "use_dropout": True,
                "bias": False},
            "dict_ups": {"BatchNorm": False,
                "p_dropout": 0.2,
                "use_dropout": False,
                "bias": False},
            "loss": nn.BCEWithLogitsLoss(),
            "optimizer": optim.Adam,
            "param_optimizer": {"weight_decay": None,
                               "lr": 5e-04},
            "use_scheduler": True,
            "type_scheduler": "StepLR",
            "scheduler": torch.optim.lr_scheduler.StepLR,
            "param_scheduler": {"step_size": 4,
                               "gamma": 0.1},
             "scaler": torch.cuda.amp.GradScaler(),
             "device": DEVICE
            }


experiment5 = {"param": dict_train5}
experiment5["convergence_path"] = run_training(dict_train5) 


f = open("result_exp/experiment5.pkl","wb")
pickle.dump(experiment5,f)
f.close()

## Experiment 6

In [ ]:
dict_train6 ={"save_model": False,
            "cross_val": True,
            "skip_connection": True,
            "num_epochs": 10,
            "n_splits": 2,
            "batch_size": 10,
            "scale_channel": 1,
            "dict_double_conv": {"BatchNorm": True,
                "activation": nn.ELU(inplace=True),
                "p_dropout": 0.5,
                "use_dropout": True,
                "bias": False},
            "dict_ups": {"BatchNorm": False,
                "p_dropout": 0.2,
                "use_dropout": False,
                "bias": False},
            "loss": nn.BCEWithLogitsLoss(),
            "optimizer": optim.Adam,
            "param_optimizer": {"weight_decay": None,
                               "lr": 5e-04},
            "use_scheduler": True,
            "type_scheduler": "StepLR",
            "scheduler": torch.optim.lr_scheduler.StepLR,
            "param_scheduler": {"step_size": 4,
                               "gamma": 0.1},
             "scaler": torch.cuda.amp.GradScaler(),
             "device": DEVICE
            }


experiment6 = {"param": dict_train6}
experiment6["convergence_path"] = run_training(dict_train6) 


f = open("result_exp/experiment6.pkl","wb")
pickle.dump(experiment6,f)
f.close()

In [8]:
import io

class CPU_Unpickler(pickle.Unpickler):
    def find_class(self, module, name):
        if module == 'torch.storage' and name == '_load_from_bytes':
            return lambda b: torch.load(io.BytesIO(b), map_location='cpu')
        else:
            return super().find_class(module, name)



f = open(f"result_exp/experiment{2}bis.pkl","rb")
previous_exp = CPU_Unpickler(f).load()
f.close()

prev_exp_last_val_F1 = 0.5*(np.array(previous_exp["convergence_path"]["Kfold 0"]["val_F1"]) + np.array(previous_exp["convergence_path"]["Kfold 1"]["val_F1"]))
exp5_last_val_F1 = 0.5*(np.array(exp5["convergence_path"]["Kfold 0"]["val_F1"]) + np.array(exp5["convergence_path"]["Kfold 1"]["val_F1"]))
exp6_last_val_F1 = 0.5*(np.array(exp6["convergence_path"]["Kfold 0"]["val_F1"]) + np.array(exp6["convergence_path"]["Kfold 1"]["val_F1"]))

best_val_F1 = 0

if prev_exp_last_val_F1 > max(exp5_last_val_F1,exp6_last_val_F1):
    use_drop = False
    best_val_F1 = prev_exp_last_val_F1
else:
    use_drop = True
    if exp5_last_val_F1>exp6_last_val_F1:
        p = 0.2
        best_val_F1 =  exp5_last_val_F1
    else:
        p = 0.5
        best_val_F1 =  exp6_last_val_F1

NameError: name 'exp5' is not defined

## Experiment 7

In [ ]:
dict_train7 ={"save_model": False,
            "cross_val": True,
            "skip_connection": True,
            "num_epochs": 10,
            "n_splits": 2,
            "batch_size": 1,
            "scale_channel": 1,
            "dict_double_conv": {"BatchNorm": True,
                "activation": nn.ELU(inplace=True),
                "p_dropout": p,
                "use_dropout": use_drop,
                "bias": False},
            "dict_ups": {"BatchNorm": False,
                "p_dropout": 0.2,
                "use_dropout": False,
                "bias": False},
            "loss": nn.BCEWithLogitsLoss(),
            "optimizer": optim.Adam,
            "param_optimizer": {"weight_decay": None,
                               "lr": 5e-04},
            "use_scheduler": True,
            "type_scheduler": "StepLR",
            "scheduler": torch.optim.lr_scheduler.StepLR,
            "param_scheduler": {"step_size": 4,
                               "gamma": 0.1},
             "scaler": torch.cuda.amp.GradScaler(),
             "device": DEVICE
            }


experiment7 = {"param": dict_train7}
experiment7["convergence_path"] = run_training(dict_train7) 


f = open("result_exp/experiment7.pkl","wb")
pickle.dump(experiment7,f)
f.close()

## Experiment 8

In [ ]:
dict_train8 ={"save_model": False,
            "cross_val": True,
            "skip_connection": True,
            "num_epochs": 10,
            "n_splits": 2,
            "batch_size": 20,
            "scale_channel": 1,
            "dict_double_conv": {"BatchNorm": True,
                "activation": nn.ELU(inplace=True),
                "p_dropout": p,
                "use_dropout": use_drop,
                "bias": False},
            "dict_ups": {"BatchNorm": False,
                "p_dropout": 0.2,
                "use_dropout": False,
                "bias": False},
            "loss": nn.BCEWithLogitsLoss(),
            "optimizer": optim.Adam,
            "param_optimizer": {"weight_decay": None,
                               "lr": 5e-04},
            "use_scheduler": True,
            "type_scheduler": "StepLR",
            "scheduler": torch.optim.lr_scheduler.StepLR,
            "param_scheduler": {"step_size": 4,
                               "gamma": 0.1},
             "scaler": torch.cuda.amp.GradScaler(),
             "device": DEVICE
            }


experiment8 = {"param": dict_train8}
experiment8["convergence_path"] = run_training(dict_train8) 


f = open("result_exp/experiment8.pkl","wb")
pickle.dump(experiment8,f)
f.close()

In [ ]:
exp7_last_val_F1 = 0.5*(np.array(exp7["convergence_path"]["Kfold 0"]["val_F1"]) + np.array(exp7["convergence_path"]["Kfold 1"]["val_F1"]))
exp8_last_val_F1 = 0.5*(np.array(exp8["convergence_path"]["Kfold 0"]["val_F1"]) + np.array(exp8["convergence_path"]["Kfold 1"]["val_F1"]))


if best_val_F1 > max(exp7_last_val_F1,exp6_last_val_F1):
        new_batch_size = 10
    best_val_F1 = best_val_F1
elif exp7_last_val_F1>max(best_val_F1,exp8_last_val_F1):
        new_batch_size = 1
        best_val_F1 =  exp7_last_val_F1
elif exp8_last_val_F1>max(best_val_F1,exp7_last_val_F1):
        new_batch_size = 20
        best_val_F1 =  exp8_last_val_F1

## Experiment 9

In [15]:
dict_train9 ={"save_model": False,
            "cross_val": True,
            "skip_connection": True,
            "num_epochs": 10,
            "n_splits": 2,
            "batch_size": new_batch_size,
            "scale_channel": 1,
            "dict_double_conv": {"BatchNorm": True,
                "activation": nn.ELU(inplace=True),
                "p_dropout": p,
                "use_dropout": use_drop,
                "bias": False},
            "dict_ups": {"BatchNorm": False,
                "p_dropout": 0.2,
                "use_dropout": False,
                "bias": False},
            "loss": nn.BCEWithLogitsLoss(),
            "optimizer": optim.Adam,
            "param_optimizer": {"weight_decay": 1e-04,
                               "lr": 5e-04},
            "use_scheduler": True,
            "type_scheduler": "StepLR",
            "scheduler": torch.optim.lr_scheduler.StepLR,
            "param_scheduler": {"step_size": 4,
                               "gamma": 0.1},
             "scaler": torch.cuda.amp.GradScaler(),
             "device": DEVICE
            }


experiment9 = {"param": dict_train9}
experiment9["convergence_path"] = run_training(dict_train9) 


f = open("result_exp/experiment9.pkl","wb")
pickle.dump(experiment9,f)
f.close()

-------------------------- 1 fold --------------------------


  0%|                                                  | 0/1100 [00:00<?, ?it/s]

(280, 280)


  0%|                              | 1/1100 [00:01<19:47,  1.08s/it, loss=0.901]

(280, 280)


...

(280, 280)


  7%|██▏                           | 81/1100 [01:05<14:04,  1.21it/s, loss=0.59]

(280, 280)


  7%|██▏                           | 81/1100 [01:06<13:51,  1.23it/s, loss=0.59]


KeyboardInterrupt: 

## Experiment 10

In [ ]:
dict_train10 ={"save_model": False,
            "cross_val": True,
            "skip_connection": True,
            "num_epochs": 10,
            "n_splits": 2,
            "batch_size": new_batch_size,
            "scale_channel": 1,
            "dict_double_conv": {"BatchNorm": True,
                "activation": nn.ELU(inplace=True),
                "p_dropout": p,
                "use_dropout": use_drop,
                "bias": False},
            "dict_ups": {"BatchNorm": False,
                "p_dropout": 0.2,
                "use_dropout": False,
                "bias": False},
            "loss": nn.BCEWithLogitsLoss(),
            "optimizer": optim.Adam,
            "param_optimizer": {"weight_decay": 1e-02,
                               "lr": 5e-04},
            "use_scheduler": True,
            "type_scheduler": "StepLR",
            "scheduler": torch.optim.lr_scheduler.StepLR,
            "param_scheduler": {"step_size": 4,
                               "gamma": 0.1},
             "scaler": torch.cuda.amp.GradScaler(),
             "device": DEVICE
            }


experiment10 = {"param": dict_train10}
experiment10["convergence_path"] = run_training(dict_train10) 


f = open("result_exp/experiment10.pkl","wb")
pickle.dump(experiment10,f)
f.close()